In [1]:
from logikon.analysts.classifier_queries import dialectic_relations, most_confirmed, most_disconfirmed
from logikon.backends.classifier import HfClassifier


In [2]:
import os

import dotenv

dotenv.load_dotenv("../.env")
"HUGGINGFACEHUB_API_TOKEN" in os.environ or print("Please set HUGGINGFACEHUB_API_TOKEN in .env")

True

In [3]:
kwargs = {
    "model_id": "MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli",
    "inference_server_url": "https://api-inference.huggingface.co/models/MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli",
    "api_key": os.environ.get("HUGGINGFACEHUB_API_TOKEN"),
    "batch_size": 4,
}
classifier = HfClassifier(**kwargs)


In [4]:
classifier

In [5]:
await classifier(
    inputs="Hi, I recently bought a device from your company but it is not working as advertised and I would like to get reimbursed!",
    classes_verbalized=["refund", "legal", "faq"],
    hypothesis_template="This text is about {}.",
    batch_size=1,
)

[HfClassification(sequence='Hi, I recently bought a device from your company but it is not working as advertised and I would like to get reimbursed!', labels=['refund', 'legal', 'faq'], scores=[0.9867447018623352, 0.0077674612402915955, 0.005487882997840643])]

In [6]:
from logikon.schemas.pros_cons import Claim

await dialectic_relations(
        arguments=[
            Claim(label="Health", text="Meat is an important source of protein"),
        ],
        claims=[
            Claim(label="Vegan", text="We all should be vegans"),
        ],
        classifier=classifier,
    )

[MultipleChoiceResult(probs={'support': 0.21592001616954803, 'attack': 0.5212746858596802, 'neutral': 0.2628052830696106}, label_max='attack', idx_max=1, choices=['support', 'attack', 'neutral'])]

In [7]:
from logikon.schemas.pros_cons import Claim

arguments=[
    Claim(label="Health", text="Egg is an important source of protein"),
    Claim(label="Taste", text="Eggs taste wonderful."),
#    Claim(label="Taste", text="Eggs taste wonderful."),
]
claims=[
    [Claim(label="Vegan", text="We all should be vegans"),Claim(label="Veggie", text="We all should be vegetarians")],
    [Claim(label="Vegan", text="We all should be vegans"),Claim(label="Veggie", text="We all should be vegetarians")],
#    [Claim(label="Meat", text="We may eat as much meat as we like"),Claim(label="Veggie", text="We all should be vegetarians")],
]


await most_confirmed(
        arguments=arguments,
        claims=claims,
        classifier=classifier,
    )

[MultipleChoiceResult(probs={'Vegan': 0.4974468946456909, 'Veggie': 0.5025530457496643}, label_max='Veggie', idx_max=1, choices=[Claim(text='We all should be vegans', label='Vegan'), Claim(text='We all should be vegetarians', label='Veggie')]),
 MultipleChoiceResult(probs={'Vegan': 0.47633492946624756, 'Veggie': 0.5236650705337524}, label_max='Veggie', idx_max=1, choices=[Claim(text='We all should be vegans', label='Vegan'), Claim(text='We all should be vegetarians', label='Veggie')])]

In [8]:
await most_disconfirmed(
        arguments=arguments,
        claims=claims,
        classifier=classifier,
    )

[MultipleChoiceResult(probs={'Vegan': 0.5780022144317627, 'Veggie': 0.4219978451728821}, label_max='Vegan', idx_max=0, choices=[Claim(text='We all should be vegans', label='Vegan'), Claim(text='We all should be vegetarians', label='Veggie')]),
 MultipleChoiceResult(probs={'Vegan': 0.6120317578315735, 'Veggie': 0.3879682719707489}, label_max='Vegan', idx_max=0, choices=[Claim(text='We all should be vegans', label='Vegan'), Claim(text='We all should be vegetarians', label='Veggie')])]

In [8]:
result

192